In [30]:
import pandas as pd
import MDAnalysis as mda
from utils import utils
import sbmlcore
import os

This notebook generates the figures in the paper 'Predicting rifampicin resistance in M. tuberculosis using machine learning informed by protein structural and chemical features.'

Authors:
- Charlotte Lynch
- Dylan Adlard
- Philip Fowler

# Data Preperation - positions and labels

Need to create a dataset of mutations with associated phenotypes from CRypTIC tables

#### Data import

In [31]:
#import data
pheno_df = pd.read_pickle('./data/tables/cryptic/DST_MEASUREMENTS-rifamycins.pkl.gz')
mut_df = pd.read_pickle('./data/tables/cryptic/MUTATIONS-rnap.pkl.gz')

#### Prepare phenotypes df

In [32]:
#filter for rifampicin phyenotypes and drop duplicates
pheno_df = pheno_df.loc[pheno_df.DRUG=='RIF']
pheno_df = pheno_df.drop_duplicates(subset=['UNIQUEID'], keep='first')

#### Prepare mutations df

In [33]:
#filter for non-synonymous solo snps in rpoB that were sequenced with FRS >= 0.9

#Note, this assues minor alleles in rpoB are not clinically significant for rifampicin resistance
mut_df = mut_df[(~mut_df['IS_NULL']) & (~mut_df['IS_SYNONYMOUS']) & (mut_df['IN_CDS'])&(mut_df['IS_FILTER_PASS'])]
#Note, this uses the defintion of solos as variants that exist in isolation in rpoB (but can have mutations in other RNAP genes)
mut_df = mut_df[(mut_df.IS_SNP)&(mut_df.GENE=='rpoB')]
#Filter for solos
mut_df.drop_duplicates(subset=['UNIQUEID'], keep=False, inplace=True)

#remove phylogenetic mutations
mut_df[~mut_df['MUTATION'].isin(['G876G', 'C66T', 'D103D', 'G86A', 'E639D'])]

#insert segid column
mut_df['segid'] = 'C'

#The pdb file also attenuates the rpoB chain at position 22 and position 1147
mut_df['resid'] = [int(i[1:-1]) for i in mut_df['MUTATION']]
mut_df = mut_df[(mut_df['resid']>=22) & (mut_df['resid']<=1147)]

#### Merge mutations and phenotypes dataframes




In [34]:
pheno_df.set_index('UNIQUEID', verify_integrity=True, inplace = True)
mut_df.set_index('UNIQUEID', verify_integrity=True, inplace=True)
merged = mut_df.join(pheno_df[['PHENOTYPE']],how='inner')

#### Phenotype the  mutations

Rather than training on all samples, we will train on unique mutations. However, many mutations appear more than once, and therefore a consensus phenotype has to be generated for these mutations to use as the label. 

Charlotte has decided to use an arbitrary 50% proportion of resistant samples for mutations that occur 3 or more times.

Are we sure we want to do this? This is saying if a mutation is R more than 50% of the time, its classified as resistant - this doesn't make any statistical sense... And doesn't make much sense given we can generate a new, accurate catalogue for rifampicin where the variant labels are grounded in at least some logic...

In [35]:
#determine R and S occurence counts for each mutation
ct = pd.crosstab([merged.MUTATION, merged.segid, merged.resid], merged.PHENOTYPE)
ct["total"] = ct.sum(axis=1)
ct["prob_R"] = ct["R"] / ct["total"]
ct["prob_S"] = ct["S"] / ct["total"]


def calculate_RS(row):
    if row["total"] >= 3:
        return "R" if row["prob_R"] > 0.5 else "S" if row["prob_S"] > 0.5 else "U"
    return "R" if row["R"] == row["total"] else "S" if row["S"] == row["total"] else "U"

#threshold R vs S proportion at 50% to call phenotype for mutations with n >= 3
ct["PHENOTYPE"] = ct.apply(calculate_RS, axis=1)
ct = ct[ct["PHENOTYPE"].isin(["R", "S"])]

ct.reset_index(inplace=True)
ct.sort_values("total", ascending=False)

PHENOTYPE,MUTATION,segid,resid,I,R,S,U,total,prob_R,prob_S,PHENOTYPE
249,S450L,C,450,0,5617,112,271,6000,0.936167,0.018667,R
44,D435V,C,435,0,726,20,29,775,0.936774,0.025806,R
120,H445D,C,445,0,383,8,14,405,0.945679,0.019753,R
129,H445Y,C,445,0,315,5,20,340,0.926471,0.014706,R
166,L452P,C,452,0,80,57,11,148,0.540541,0.385135,R
...,...,...,...,...,...,...,...,...,...,...,...
61,D997E,C,997,0,0,1,0,1,0.000000,1.000000,S
182,M707T,C,707,0,1,0,0,1,1.000000,0.000000,R
183,N135I,C,135,0,0,1,0,1,0.000000,1.000000,S
60,D993G,C,993,0,0,1,0,1,0.000000,1.000000,S


#### Generate sample df

In [36]:
# Create final DataFrame from ct
df = pd.DataFrame({
    'mutation': ct.MUTATION,
    'segid': ct['segid'],
    'PHENOTYPE': ct['PHENOTYPE'],
    'resid': ct['resid']
})

# Optionally reset index if needed (though it may not be necessary)
df.reset_index(drop=True, inplace=True)
df.to_csv('./data/tables/generated/rpoB_solos.csv')
df


,mutation,segid,PHENOTYPE,resid
0,A1002P,C,S,1002
1,A286V,C,R,286
2,A29T,C,S,29
3,A29V,C,S,29
4,A334D,C,S,334
...,...,...,...,...
302,V895F,C,S,895
303,V946G,C,S,946
304,Y308C,C,R,308
305,Y572C,C,S,572


In [37]:
# R vs S weighting:
print ('Proportion of Resisant mutations:', df.PHENOTYPE.value_counts())

Proportion of Resisant mutations: PHENOTYPE
S    259
R     48
Name: count, dtype: int64


This dataframe basically acts as a catalogue, with unique mutations and their associated labels.

# Mutation mapping onto RNAP and rpoB structures

Where on the structure of rpoB do these mutation exist?

In [38]:
utils.map_mut2pdb(df, './data/pdb/5uh6.pdb', './data/pdb/rnap_mutations.pdb')

/Users/dylanadlard/miniforge3/envs/rif_manuscript/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/Users/dylanadlard/miniforge3/envs/rif_manuscript/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


TO DO: generate figure 1 and display the png here.

# Data preperation - Feature engineering

We have a table of mutations and their labels. Next step is to generate a global feature set for each mutation using sbmlcore.

In [39]:
features = sbmlcore.FeatureDataset(df, protein="RNAP", species="M. tuberculosis", gene="rpoB")

#### Generate amino acid features

In [40]:
#change in amino acid volume on mutation
volume = sbmlcore.AminoAcidVolumeChange()
#change in amino acid hydropathy on mutation
hydropathy = sbmlcore.AminoAcidHydropathyChangeKyteDoolittle()
#change in amino acid molecular weight on mutation
mw = sbmlcore.AminoAcidMWChange()
#change in amino acid isoelectric point on mutation
pi = sbmlcore.AminoAcidPiChange()
#change in residue-environment similarity
rogov = sbmlcore.AminoAcidRogovChange()

features.add_feature([volume, hydropathy, mw, pi, rogov])

#### Generate distance-based features

In [41]:
#distance from each residue to rifampicin
rif_distance = sbmlcore.StructuralDistances('./data/pdb/5uh6.pdb','resname RFP', 'Rif_distance', infer_masses=True, offsets = {"C": -6})
#distance from each residue to the magnesium ion
mg_distance = sbmlcore.StructuralDistances('./data/pdb/5uh6.pdb','resname MG', 'Mg_distance', infer_masses=True, offsets = {"C": -6})
#distance from each residue to the first zinc ion
zn1_distance = sbmlcore.StructuralDistances('./data/pdb/5uh6.pdb','index 26082 and resname ZN', 'Zn1_distance', infer_masses=True, offsets = {"C": -6})
#distance from each residue to the second zinc ion
zn2_distance = sbmlcore.StructuralDistances('./data/pdb/5uh6.pdb','index 26083 and resname ZN', 'Zn2_distance', infer_masses=True, offsets = {"C": -6})
#distance from each residue to the antisense DNA strand
antisense_p_distance = sbmlcore.StructuralDistances('./data/pdb/5uh6.pdb','segid G and name P', 'antisense_P_distance', infer_masses=True, offsets = {"C": -6})
#distance from each residue to the coding DNA strand
sense_p_distance = sbmlcore.StructuralDistances('./data/pdb/5uh6.pdb','segid H and name P', 'sense_P_distance', infer_masses=True, offsets = {"C": -6})
#distance from each residue to the crystallised mRNA molecule
rna_distance = sbmlcore.StructuralDistances('./data/pdb/5uh6.pdb','segid I', 'RNA_distance', infer_masses=True, offsets = {"C": -6})

features.add_feature([rif_distance, mg_distance, zn1_distance, zn2_distance, antisense_p_distance, sense_p_distance, rna_distance])

####  Generate structure-related features

In [42]:
# use stride to assign secondary structure to each residue
stride = sbmlcore.Stride("./data/pdb/5uh6-peptide-only.pdb", offsets={'A': 0, 'B': 0, 'C':-6, 'D':0, 'E':0, 'F':0})
# use freeSASA to calculate solvent accessible surface area of each residue
freesasa = sbmlcore.FreeSASA("./data/pdb/5uh6.pdb", offsets = {'C':-6})
# use SNAP2 to predict functional perturbation for each mutation (in reality just adding pre-generated feature to df)
snap2 = sbmlcore.SNAP2("./data/stride/5uh6-complete.csv", offsets={'A': 0, 'B': 0, 'C':-6, 'D':0, 'E':0, 'F':0})
# use deepddg to predict change in stability on mutation
deepddg = sbmlcore.DeepDDG("./data/ddg/5uh6.ddg", offsets={'A': 0, 'B': 0, 'C':-6, 'D':0, 'E':0, 'F':0})
# use rasp to predict change in stability on mutation
rasp = sbmlcore.RaSP("./data/rasp/cavity_pred_5uh6_C.csv", offsets = {'C':-6})
# extract temperature factors from pdb file and add to df
temp = sbmlcore.TempFactors("./data/pdb/5uh6.pdb", offsets={'A': 0, 'B': 0, 'C':-6, 'D':0, 'E':0, 'F':0})

features.add_feature([stride, freesasa, snap2, deepddg, rasp, temp])

/Users/dylanadlard/miniforge3/envs/rif_manuscript/lib/python3.12/site-packages/sbmlcore/ExternalCode.py:313: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  snap2_df['Expected Accuracy'].replace(to_replace='%', value='', regex=True, inplace=True)


#### Clean up

In [43]:
#annotate secondary structures with integer codes
features.df['secondary_structure_codes'] = pd.Categorical(features.df.secondary_structure, categories=features.df.secondary_structure.unique()).codes
features.df = features.df.drop(columns=["secondary_structure","secondary_structure_long","B","C","E","G","H","T"])
#drop superfluous columns
features.df = features.df.drop(columns=["residue_sasa","snap2_accuracy","rasp_wt_nlf","rasp_mt_nlf","rasp_score_ml"])
#rename columns
features.df = features.df.rename(columns={"d_Pi":"d_pi", "rasp_score_ml_fermi":"rasp_score", "secondary_structure_codes":"secondary_structure"})

features.df.to_csv('./data/tables/generated/features_dataset.csv')
df = features.df
df

,segid,mutation,PHENOTYPE,d_volume,d_hydropathy_KD,d_MW,d_pi,d_rogov,Rif_distance,Mg_distance,...,phi,psi,n_hbond_acceptors,n_hbond_donors,SASA,snap2_score,deep_ddG,rasp_score,temp_factor,secondary_structure
0,C,A1002P,S,24.1,-3.4,26.0,0.30,0.020,45.415055,31.783345,...,-155.77,164.84,1.0,1.0,7.268116,36,-2.927,0.800033,36.009998,0
1,C,A286V,R,51.4,2.4,28.0,-0.04,0.232,47.831892,64.028439,...,-98.83,-17.70,0.0,1.0,0.000000,-31,-0.523,0.222401,121.500000,1
2,C,A29T,S,27.5,-2.5,30.0,-0.40,0.248,34.306893,40.030977,...,-91.15,161.38,0.0,0.0,34.753706,-81,-1.142,0.328659,32.820000,2
3,C,A29V,S,51.4,2.4,28.0,-0.04,0.232,34.306893,40.030977,...,-91.15,161.38,0.0,0.0,34.753706,-76,-1.056,0.284229,32.820000,2
4,C,A334D,S,22.5,-5.3,44.0,-3.23,0.029,43.228183,56.772771,...,-67.17,-16.26,1.0,1.0,0.786252,-54,-2.734,0.369132,102.089996,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,C,V895F,S,49.9,-1.4,48.1,-0.48,0.345,41.296670,28.986757,...,-59.98,-22.89,0.0,0.0,35.841417,-1,-1.014,0.287163,37.750000,3
303,C,V946G,S,-79.9,-4.6,-42.0,0.01,-0.080,53.471656,48.889279,...,-77.65,122.58,0.0,0.0,86.193134,-81,-0.444,0.424508,68.440002,2
304,C,Y308C,R,-85.1,3.8,-60.0,-0.59,-0.199,33.128330,49.415429,...,-65.00,-35.81,1.0,1.0,70.874065,68,-1.361,0.467171,76.190002,1
305,C,Y572C,S,-85.1,3.8,-60.0,-0.59,-0.199,37.911061,45.458752,...,-129.88,172.92,1.0,1.0,27.871431,35,-1.543,0.563006,61.090000,0
